In [134]:
# General DS libs
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

# Libs for setting up DB conn
import psycopg2
import os
from dotenv import load_dotenv

In [135]:
# Set up DB conn
load_dotenv()

conn = psycopg2.connect(
    dbname=os.getenv("PG_DB"),
    user=os.getenv("PG_USER"),
    password=os.getenv("PG_PASSWORD"),
    host=os.getenv("PG_HOST"),
    port=os.getenv("PG_PORT"),
    sslmode="require"
)

In [136]:
posts_df = pd.read_sql("SELECT * FROM reddit_posts", conn)
comments_df = pd.read_sql("SELECT * FROM reddit_comments", conn)
conn.close()

/tmp/ipykernel_8441/1077863026.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  posts_df = pd.read_sql("SELECT * FROM reddit_posts", conn)
/tmp/ipykernel_8441/1077863026.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  comments_df = pd.read_sql("SELECT * FROM reddit_comments", conn)


To test the algorithm, I will test only the posts & comments from 2025-07-10 00:00:00 AM to 2025-07-10 2025-07-10 23:59:59 AM SGT

In [137]:
import datetime
begin_timestamp = np.int64(datetime.datetime(2025, 7, 9, 23, 0, 0).timestamp())    # I am currently in GMT+07
end_timestamp = np.int64(datetime.datetime(2025, 7, 10, 22, 59, 59).timestamp()) 
begin_timestamp, end_timestamp

(np.int64(1752076800), np.int64(1752163199))

In [138]:
posts_df = posts_df[(posts_df["created_utc"] >= begin_timestamp) & (posts_df["created_utc"] <= end_timestamp)]
posts_df = posts_df.reset_index(drop=True)
posts_df.head()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers
0,1lweoyh,Bitcoin,111111 make a wish!,,noddingacquaintance,1752159048,27,27,6,,n2dgavm: This is spiritual awakening number ||...,0.91,
1,1lwegwg,Bitcoin,Bitcoin Optech Newsletter #361 Recap Podcast,"Sanket Kanjalkar, Jonas Nick, Tadge Dryja, Ste...",bitschmidty,1752158493,0,0,0,,,0.50,
2,1lwee8g,Bitcoin,Bruh,Lmao can you imagine.,Areuregarded,1752158315,308,308,63,,n2de5yo: Vietnamese Dong will getcha every tim...,0.93,
3,1lwee26,Bitcoin,An All-Time High Means a Free Bitcoin Book,"Hey all,\n\nTo celebrate Bitcoin’s all-time hi...",Robinandai,1752158304,1,1,4,,n2dk598: link not working || n2duqv4: Give fre...,0.60,
4,1lwdvht,Bitcoin,A yearly Bitcoin race to a public wallet: the ...,"This idea was inspired by the Nobel Prize, but...",Pomonoli,1752157034,0,0,3,,n2decxm: > This idea was inspired by the Nobel...,0.50,


In [139]:
posts_df.shape

(82, 13)

In [140]:
posts_df[posts_df["body"] == ""].shape[0], posts_df[posts_df["body"].isna()].shape[0]

(28, 0)

Note here that if there's no body text in a post, it's denoted as empty string "" instead of NaN

In [141]:
comments_df = comments_df[(comments_df["created_utc"] >= begin_timestamp) & (comments_df["created_utc"] <= end_timestamp)]
comments_df = comments_df.reset_index(drop=True)
comments_df.head()

,comment_id,post_id,author,body,score,created_utc
0,n2dgavm,1lweoyh,birjy,This is spiritual awakening number,3,1752159137
1,n2dgugu,1lweoyh,No_Key_Sentence,Who would have thought we are going to see suc...,3,1752159291
2,n2dn8nm,1lweoyh,Quokky-Axolotl7388,I wish to add another 1,1,1752161067
3,n2dqzdm,1lweoyh,StaticAutomatic202,"I wish for 222, 222",1,1752162122
4,n2ds97n,1lweoyh,24Bayne24,Hi Keith 🙏🏼,1,1752162481


In [142]:
comments_df.shape

(828, 6)

# 1. Mandatory Features (real data)

## 1.1. Data Cleaning

In [143]:
import re
import contractions
import emoji

def clean_text(text):
    """# General data cleaning function for subreddit texts, including post's title, body and comments"""
    # If text is empty, leave it be
    if text == "":
        return ""

    # Replace multiple whitespaces with just one
    text = re.sub(r"\s+", " ", text)

    # Convert all emojis to textual representation
    text = emoji.demojize(text)

    # Replace URLs with tag <URL>
    text = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", 
                  "<URL>", text)
    
	# Expand contractions in the text
    text = contractions.fix(text)

    return text

In [144]:
posts_df["cleaned_title"] = posts_df["title"].apply(clean_text)
posts_df["cleaned_body"] = posts_df["body"].apply(clean_text)
comments_df["cleaned_comment"] = comments_df["body"].apply(clean_text)

In [145]:
posts_df.head()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers,cleaned_title,cleaned_body
0,1lweoyh,Bitcoin,111111 make a wish!,,noddingacquaintance,1752159048,27,27,6,,n2dgavm: This is spiritual awakening number ||...,0.91,,111111 make a wish!,
1,1lwegwg,Bitcoin,Bitcoin Optech Newsletter #361 Recap Podcast,"Sanket Kanjalkar, Jonas Nick, Tadge Dryja, Ste...",bitschmidty,1752158493,0,0,0,,,0.50,,Bitcoin Optech Newsletter #361 Recap Podcast,"Sanket Kanjalkar, Jonas Nick, Tadge Dryja, Ste..."
2,1lwee8g,Bitcoin,Bruh,Lmao can you imagine.,Areuregarded,1752158315,308,308,63,,n2de5yo: Vietnamese Dong will getcha every tim...,0.93,,Bruh,Lmao can you imagine.
3,1lwee26,Bitcoin,An All-Time High Means a Free Bitcoin Book,"Hey all,\n\nTo celebrate Bitcoin’s all-time hi...",Robinandai,1752158304,1,1,4,,n2dk598: link not working || n2duqv4: Give fre...,0.60,,An All-Time High Means a Free Bitcoin Book,"Hey all, To celebrate Bitcoin’s all-time high,..."
4,1lwdvht,Bitcoin,A yearly Bitcoin race to a public wallet: the ...,"This idea was inspired by the Nobel Prize, but...",Pomonoli,1752157034,0,0,3,,n2decxm: > This idea was inspired by the Nobel...,0.50,,A yearly Bitcoin race to a public wallet: the ...,"This idea was inspired by the Nobel Prize, but..."


In [146]:
comments_df.head()

,comment_id,post_id,author,body,score,created_utc,cleaned_comment
0,n2dgavm,1lweoyh,birjy,This is spiritual awakening number,3,1752159137,This is spiritual awakening number
1,n2dgugu,1lweoyh,No_Key_Sentence,Who would have thought we are going to see suc...,3,1752159291,Who would have thought we are going to see suc...
2,n2dn8nm,1lweoyh,Quokky-Axolotl7388,I wish to add another 1,1,1752161067,I wish to add another 1
3,n2dqzdm,1lweoyh,StaticAutomatic202,"I wish for 222, 222",1,1752162122,"I wish for 222, 222"
4,n2ds97n,1lweoyh,24Bayne24,Hi Keith 🙏🏼,1,1752162481,Hi Keith :folded_hands_medium-light_skin_tone:


## 1.2. Run the sentiment analyzer

In [147]:
import torch

In [ ]:
# This is to delete CUDA cache
try:
    del tokenizer
except:
    print("tokenizer already deleted")

try:
    del model
except:
    print("model already deleted")

import gc
torch.cuda.empty_cache()
gc.collect()

2209

In [149]:
# Define label-id mappings
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [150]:
from transformers import BertForSequenceClassification, BertTokenizerFast, BertConfig
model_name = "ProsusAI/finbert"

tokenizer = BertTokenizerFast.from_pretrained(
    model_name,
    use_fast=True,
    padding_side="left",
    padding_token='[PAD]',
)

config = BertConfig.from_pretrained(
    model_name,
    num_labels=3,
    label2id=label2id,
    id2label=id2label,
)

model = BertForSequenceClassification.from_pretrained(
    model_name,
    config=config,
    device_map="auto",
)

In [151]:
from transformers import pipeline

finbert_classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    top_k=3
)

Device set to use cuda:0


In [152]:
posts_df["cleaned_body"].isna().sum(), posts_df["cleaned_title"].isna().sum(), comments_df["cleaned_comment"].isna().sum()

(np.int64(0), np.int64(0), np.int64(0))

In [153]:
# We only want to apply sentiment analysis on non-na entries
non_na_post_body = posts_df[~(posts_df["cleaned_body"] == "")]
non_na_post_body = non_na_post_body[["post_id", "cleaned_body"]]
non_na_post_body.head()

,post_id,cleaned_body
1,1lwegwg,"Sanket Kanjalkar, Jonas Nick, Tadge Dryja, Ste..."
2,1lwee8g,Lmao can you imagine.
3,1lwee26,"Hey all, To celebrate Bitcoin’s all-time high,..."
4,1lwdvht,"This idea was inspired by the Nobel Prize, but..."
9,1lwavum,I feel blessed to have discovered Bitcoin at a...


In [154]:
# Apply pipeline on title, selftext and comments
title_sentiments = finbert_classifier(
    posts_df["cleaned_title"].to_list(),
    truncation=True,
    max_length=512,
)
post_body_sentiments = finbert_classifier(
    non_na_post_body["cleaned_body"].to_list(),
    truncation=True,
    max_length=512,
)
comment_sentiments = finbert_classifier(
    comments_df["cleaned_comment"].to_list(),
    truncation=True,
    max_length=512,
)

In [155]:
posts_df.tail()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers,cleaned_title,cleaned_body
77,1lwfyv3,Bitcoin,Orange Is The New Gold,,OkEstablishment7095,1752162089,0,0,3,,n2dvfxj: \#DownvoteAI || n2drw02: Whys there s...,0.33,,Orange Is The New Gold,
78,1lwfn10,Bitcoin,Whats the difference between Satoshi’s keys / ...,I heard that Satoshi's Bitcoins are easier to ...,Ok-Depth608,1752161293,0,0,8,,n2dptoh: The way wallets and keys were created...,0.50,,What Is the difference between Satoshi’s keys ...,I heard that Satoshi's Bitcoins are easier to ...
79,1lwf4bq,Bitcoin,Getting 10% back in bitcoin on a credit card...,https://x.com/BitcoinGrower/status/19433254699...,micro23,1752160076,0,0,5,,"n2dkqt1: ""Not trying to promote it."" ...ends u...",0.20,,Getting 10% back in bitcoin on a credit card...,<URL> I do not know how they are doing it but ...
80,1lwez7h,Bitcoin,Bullish News,Form 1099-DA was designed to require crypto br...,Available-Mission918,1752159740,9,9,1,,n2dmt4x: I think they're just making tweaks to...,0.91,,Bullish News,Form 1099-DA was designed to require crypto br...
81,1lwewhy,Bitcoin,We doing fiat coasters now?,Iykyk,edwardblilley,1752159568,4,4,0,,,0.70,,We doing fiat coasters now?,Iykyk


In [156]:
# Attach the scores back the the dataframe where they come from
posts_df = pd.concat([posts_df, pd.DataFrame({"title_score_list" : title_sentiments})], axis=1)
non_na_post_body = pd.concat([non_na_post_body, pd.DataFrame({"post_body_score_list" : post_body_sentiments})], axis=1)
comments_df = pd.concat([comments_df, pd.DataFrame({"comment_score_list" : comment_sentiments})], axis=1)

In [157]:
posts_df.tail()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers,cleaned_title,cleaned_body,title_score_list
77,1lwfyv3,Bitcoin,Orange Is The New Gold,,OkEstablishment7095,1752162089,0,0,3,,n2dvfxj: \#DownvoteAI || n2drw02: Whys there s...,0.33,,Orange Is The New Gold,,"[{'label': 'positive', 'score': 0.898133695125..."
78,1lwfn10,Bitcoin,Whats the difference between Satoshi’s keys / ...,I heard that Satoshi's Bitcoins are easier to ...,Ok-Depth608,1752161293,0,0,8,,n2dptoh: The way wallets and keys were created...,0.50,,What Is the difference between Satoshi’s keys ...,I heard that Satoshi's Bitcoins are easier to ...,"[{'label': 'positive', 'score': 0.929334402084..."
79,1lwf4bq,Bitcoin,Getting 10% back in bitcoin on a credit card...,https://x.com/BitcoinGrower/status/19433254699...,micro23,1752160076,0,0,5,,"n2dkqt1: ""Not trying to promote it."" ...ends u...",0.20,,Getting 10% back in bitcoin on a credit card...,<URL> I do not know how they are doing it but ...,"[{'label': 'positive', 'score': 0.624392211437..."
80,1lwez7h,Bitcoin,Bullish News,Form 1099-DA was designed to require crypto br...,Available-Mission918,1752159740,9,9,1,,n2dmt4x: I think they're just making tweaks to...,0.91,,Bullish News,Form 1099-DA was designed to require crypto br...,"[{'label': 'positive', 'score': 0.776126205921..."
81,1lwewhy,Bitcoin,We doing fiat coasters now?,Iykyk,edwardblilley,1752159568,4,4,0,,,0.70,,We doing fiat coasters now?,Iykyk,"[{'label': 'positive', 'score': 0.874758601188..."


In [158]:
# Left join reddit_df and non_na_selftext based on post id
posts_df = pd.merge(posts_df, non_na_post_body[["post_id", "post_body_score_list"]],
                     how="left",
                     on="post_id")

In [159]:
posts_df.tail()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers,cleaned_title,cleaned_body,title_score_list,post_body_score_list
77,1lwfyv3,Bitcoin,Orange Is The New Gold,,OkEstablishment7095,1752162089,0,0,3,,n2dvfxj: \#DownvoteAI || n2drw02: Whys there s...,0.33,,Orange Is The New Gold,,"[{'label': 'positive', 'score': 0.898133695125...",NaN
78,1lwfn10,Bitcoin,Whats the difference between Satoshi’s keys / ...,I heard that Satoshi's Bitcoins are easier to ...,Ok-Depth608,1752161293,0,0,8,,n2dptoh: The way wallets and keys were created...,0.50,,What Is the difference between Satoshi’s keys ...,I heard that Satoshi's Bitcoins are easier to ...,"[{'label': 'positive', 'score': 0.929334402084...",NaN
79,1lwf4bq,Bitcoin,Getting 10% back in bitcoin on a credit card...,https://x.com/BitcoinGrower/status/19433254699...,micro23,1752160076,0,0,5,,"n2dkqt1: ""Not trying to promote it."" ...ends u...",0.20,,Getting 10% back in bitcoin on a credit card...,<URL> I do not know how they are doing it but ...,"[{'label': 'positive', 'score': 0.624392211437...",NaN
80,1lwez7h,Bitcoin,Bullish News,Form 1099-DA was designed to require crypto br...,Available-Mission918,1752159740,9,9,1,,n2dmt4x: I think they're just making tweaks to...,0.91,,Bullish News,Form 1099-DA was designed to require crypto br...,"[{'label': 'positive', 'score': 0.776126205921...",NaN
81,1lwewhy,Bitcoin,We doing fiat coasters now?,Iykyk,edwardblilley,1752159568,4,4,0,,,0.70,,We doing fiat coasters now?,Iykyk,"[{'label': 'positive', 'score': 0.874758601188...",NaN


In [160]:
comments_df.tail()

,comment_id,post_id,author,body,score,created_utc,cleaned_comment,comment_score_list
823,n29336t,1lvnhdg,Weak-Imagination9363,"Land like BTC is finite, and good land is actu...",5,1752096470,"Land like BTC is finite, and good land is actu...","[{'label': 'positive', 'score': 0.831269800662..."
824,n296xa6,1lvnhdg,Terhonator,Agree. This is just facts.,4,1752097597,Agree. This is just facts.,"[{'label': 'positive', 'score': 0.896991789340..."
825,n27vyoq,1lvnhdg,FixedGearJunkie,.5 BTC,12,1752084523,.5 BTC,"[{'label': 'positive', 'score': 0.925537049770..."
826,n28zl9f,1lvnhdg,Jonathaan,0.5,4,1752095470,0.5,"[{'label': 'positive', 'score': 0.877273797988..."
827,n28f7w3,1lvnhdg,Clanorr,It is not the house materials that is inflatin...,8,1752089867,It is not the house materials that is inflatin...,"[{'label': 'positive', 'score': 0.923235356807..."


In [118]:
type(comments_df["comment_score_list"][0])

list

In [119]:
posts_df["post_body_score_list"].head(30)

0     [{'label': 'positive', 'score': 0.424184828996...
1     [{'label': 'positive', 'score': 0.727404534816...
2     [{'label': 'positive', 'score': 0.940490186214...
3     [{'label': 'positive', 'score': 0.933543682098...
4     [{'label': 'positive', 'score': 0.934151411056...
5     [{'label': 'positive', 'score': 0.424184828996...
6     [{'label': 'positive', 'score': 0.424184828996...
7     [{'label': 'positive', 'score': 0.424184828996...
8     [{'label': 'positive', 'score': 0.424184828996...
9     [{'label': 'positive', 'score': 0.916810631752...
10    [{'label': 'positive', 'score': 0.759009599685...
11    [{'label': 'positive', 'score': 0.424184828996...
12    [{'label': 'positive', 'score': 0.424184828996...
13    [{'label': 'positive', 'score': 0.424184828996...
14    [{'label': 'positive', 'score': 0.914411604404...
15    [{'label': 'positive', 'score': 0.424184828996...
16    [{'label': 'positive', 'score': 0.424184828996...
17    [{'label': 'positive', 'score': 0.79803234

In [161]:
def calc_compound_score(score_list):
    """Calculate compound score of a piece of text as positive probability - negative probability"""
    # If the score list is NaN, leave it be
    if isinstance(score_list, float):
        if pd.isna(score_list):
            return np.nan

    return score_list[0]["score"] - score_list[1]["score"]

In [162]:
# Calculate compound score as pos score - neg score
posts_df["title_compound_score"] = posts_df["title_score_list"].apply(lambda x: calc_compound_score(x))
posts_df["title_compound_score"].head()

0    0.858748
1    0.888952
2    0.797769
3    0.586036
4    0.808562
Name: title_compound_score, dtype: float64

In [163]:
posts_df["post_body_compound_score"] = posts_df["post_body_score_list"].apply(lambda x: calc_compound_score(x))
comments_df["comment_compound_score"] = comments_df["comment_score_list"].apply(lambda x: calc_compound_score(x))

In [164]:
posts_df["post_body_compound_score"]

0          NaN
1     0.904300
2     0.889310
3     0.893505
4     0.857313
        ...   
77         NaN
78         NaN
79         NaN
80         NaN
81         NaN
Name: post_body_compound_score, Length: 82, dtype: float64

In [165]:
# Save the df for future work
posts_df.to_csv("posts_df_with_scores.csv", index=False)
comments_df.to_csv("comments_df_with_scores.csv", index=False)

## 1.4. Apply weights & Aggregate

There are many different kinds of weighting that may fit the needs for multiple users. Here, I apply power weighting first, with the intuition that viral posts are more important than unpopular posts in shaping sentiment, by an order greater than linear

In [166]:
# Load the df again
posts_df = pd.read_csv("posts_df_with_scores.csv")
comments_df = pd.read_csv("comments_df_with_scores.csv")

In [180]:
posts_df.tail()

,post_id,subreddit,title,body,author,created_utc,upvotes,score,num_comments,flair,comments,upvote_ratio,tickers,cleaned_title,cleaned_body,title_score_list,post_body_score_list,title_compound_score,post_body_compound_score
77,1lwfyv3,Bitcoin,Orange Is The New Gold,NaN,OkEstablishment7095,1752162089,0,0,3,NaN,n2dvfxj: \#DownvoteAI || n2drw02: Whys there s...,0.33,NaN,Orange Is The New Gold,NaN,"[{'label': 'positive', 'score': 0.898133695125...",NaN,0.811370,NaN
78,1lwfn10,Bitcoin,Whats the difference between Satoshi’s keys / ...,I heard that Satoshi's Bitcoins are easier to ...,Ok-Depth608,1752161293,0,0,8,NaN,n2dptoh: The way wallets and keys were created...,0.50,NaN,What Is the difference between Satoshi’s keys ...,I heard that Satoshi's Bitcoins are easier to ...,"[{'label': 'positive', 'score': 0.929334402084...",NaN,0.889838,NaN
79,1lwf4bq,Bitcoin,Getting 10% back in bitcoin on a credit card...,https://x.com/BitcoinGrower/status/19433254699...,micro23,1752160076,0,0,5,NaN,"n2dkqt1: ""Not trying to promote it."" ...ends u...",0.20,NaN,Getting 10% back in bitcoin on a credit card...,<URL> I do not know how they are doing it but ...,"[{'label': 'positive', 'score': 0.624392211437...",NaN,0.260446,NaN
80,1lwez7h,Bitcoin,Bullish News,Form 1099-DA was designed to require crypto br...,Available-Mission918,1752159740,9,9,1,NaN,n2dmt4x: I think they're just making tweaks to...,0.91,NaN,Bullish News,Form 1099-DA was designed to require crypto br...,"[{'label': 'positive', 'score': 0.776126205921...",NaN,0.576542,NaN
81,1lwewhy,Bitcoin,We doing fiat coasters now?,Iykyk,edwardblilley,1752159568,4,4,0,NaN,NaN,0.70,NaN,We doing fiat coasters now?,Iykyk,"[{'label': 'positive', 'score': 0.874758601188...",NaN,0.791775,NaN


In [169]:
comments_df.tail()

,comment_id,post_id,author,body,score,created_utc,cleaned_comment,comment_score_list,comment_compound_score
823,n29336t,1lvnhdg,Weak-Imagination9363,"Land like BTC is finite, and good land is actu...",5,1752096470,"Land like BTC is finite, and good land is actu...","[{'label': 'positive', 'score': 0.831269800662...",0.729140
824,n296xa6,1lvnhdg,Terhonator,Agree. This is just facts.,4,1752097597,Agree. This is just facts.,"[{'label': 'positive', 'score': 0.896991789340...",0.834709
825,n27vyoq,1lvnhdg,FixedGearJunkie,.5 BTC,12,1752084523,.5 BTC,"[{'label': 'positive', 'score': 0.925537049770...",0.885184
826,n28zl9f,1lvnhdg,Jonathaan,0.5,4,1752095470,0.5,"[{'label': 'positive', 'score': 0.877273797988...",0.800001
827,n28f7w3,1lvnhdg,Clanorr,It is not the house materials that is inflatin...,8,1752089867,It is not the house materials that is inflatin...,"[{'label': 'positive', 'score': 0.923235356807...",0.879514


In [177]:
posts_df[posts_df["score"] != posts_df["upvotes"]].shape[0]

0

As Reddit doesn't reveal the number of downvotes, it seems like score and upvotes are now one and the same. But we can still infer downvotes based on upvote_ratio if we need to

In [182]:
posts_df["weights"] = posts_df["score"].apply(lambda x: x**1.5 if x >= 1 else 1) # x**2 would be a bit too much
comments_df["weights"] = comments_df["score"].apply(lambda x: x**1.5 if x >= 1 else 1) # x**2 would be a bit too much

In [187]:
non_na_post_body = posts_df[~pd.isna(posts_df["post_body_compound_score"])]

In [188]:
# Aggregate
daily_sentiment_score = (sum(posts_df["title_compound_score"] * posts_df["weights"]) + \
                          sum(non_na_post_body["post_body_compound_score"] * non_na_post_body["weights"]) + \
                          sum(comments_df["comment_compound_score"] * comments_df["weights"])) / \
                          (sum(posts_df["weights"]) + sum(non_na_post_body["weights"]) + sum(comments_df["weights"]))
print(f"Daily sentiment score: {daily_sentiment_score:.4f}")

Daily sentiment score: 0.7409


Note: I can further apply lower weights for fresher posts as well